In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import time
import datetime

df = pd.read_csv("https://raw.githubusercontent.com/daniel-dc-cd/data_science/master/module_4_ML/data/seattle_weather_1948-2017.csv")

numrows = 25549

In [2]:
heuristic_df = pd.DataFrame({'yesterday':[0.0]*numrows,
                             'today':[0.0]*numrows,
                             'tomorrow':[0.0]*numrows,
                             'guess':[False]*numrows, #logical guess
                             'rain_tomorrow':[False]*numrows, #historical observation
                             'correct':[False]*numrows, #TRUE if your guess matches the historical observation
                             'true_positive':[False]*numrows, #TRUE If you said it would rain and it did
                             'false_positive':[False]*numrows,#TRUE If you sait id would rain and it didn't
                             'true_negative':[False]*numrows, #TRUE if you said it wouldn't rain and it didn't
                             'false_negative':[False]*numrows}) #TRUE if you said it wouldn't raing and it did

#sort columns for convience
seq = ['yesterday',
       'today',
       'tomorrow',
       'guess',
       'rain_tomorrow',
       'correct',
       'true_positive',
       'false_positive',
       'true_negative',
       'false_negative']
heuristic_df = heuristic_df.reindex(columns=seq)

In [3]:
for z in range(numrows):
    #start at time 2 in the data frame
    i = z + 2
    #pull values from the dataframe
    yesterday = df.iloc[(i-2),1]
    today = df.iloc[(i-1),1]
    tomorrow = df.iloc[i,1]
    rain_tomorrow = df.iloc[(i),1]
    
    heuristic_df.iat[z,0] = yesterday
    heuristic_df.iat[z,1] = today
    heuristic_df.iat[z,2] = tomorrow
    heuristic_df.iat[z,3] = False # set guess default to False
    heuristic_df.iat[z,4] = rain_tomorrow
    
    #example hueristic
    if today > 0.0 and yesterday > 0.0:
        heuristic_df.iat[z,3] = True
        
    if heuristic_df.iat[z,3] == heuristic_df.iat[z,4]:
        heuristic_df.iat[z,5] = True
        if heuristic_df.iat[z,3] == True:
            heuristic_df.iat[z,6] = True #true positive
        else:
            heuristic_df.iat[z,8] = True #true negative
    else:
        heuristic_df.iat[z,5] = False
        if heuristic_df.iat[z,3] == True:
            heuristic_df.iat[z,7] = True #false positive
        else:
            heuristic_df.iat[z,9] = True #false negative

In [73]:
# split data into training and testing

from sklearn.model_selection import train_test_split

X=heuristic_df
y=heuristic_df
y=y.astype('str')

X_train, X_test, y_train, y_test = train_test_split(X , y, test_size=0.2)

In [78]:
# Calculate Precision for training and test sets

# for training set
NTP_tr=X_train['true_positive'].sum()
NFP_tr=X_train['false_positive'].sum()
prec = NTP_tr/(NTP_tr+NFP_tr)
print("Training Precision:", prec)

# for testing set
NTP_te=X_test['true_positive'].sum()
NFP_te=X_test['false_positive'].sum()
prec = NTP_te/(NTP_te+NFP_te)
print("Testing Precision:", prec)

Training Precision: 0.6759113901971044
Testing Precision: 0.6671177266576455


In [81]:
# Calculate Recall for training and test sets

# for training set
NFN_tr=X_train['false_negative'].sum()
rec=NTP_tr/(NTP_tr+NFN_tr)
print("Training Recall:", rec)

# for testing set
NFN_te=X_test['false_negative'].sum()
rec=NTP_te/(NTP_te+NFN_te)
print("Testing Recall:", rec)

Training Recall: 0.4435160810346801
Testing Recall: 0.455637707948244


In [89]:
# Calculate SSE for both training and test sets


import numpy as np
from sklearn.metrics import mean_squared_error

def sse(y_true, y_pred):
    '''returns sum of squared errors (actual vs model)'''
    squared_errors = (y_true - y_pred) ** 2
    return np.sum(squared_errors)

# for training set
SSE_tr = sse(y_true=X_train['rain_tomorrow'].astype('int'), y_pred=X_train['guess'].astype('int'))
mean_SSE_tr = mean_squared_error(y_true=X_train['rain_tomorrow'].astype('int'), y_pred=X_train['guess'].astype('int'))
print("Training SSE:" ,SSE_tr)
print("Training Mean SSE:" ,mean_SSE_tr)

# for testing set
SSE_te = sse(y_true=X_test['rain_tomorrow'].astype('int'), y_pred=X_test['guess'].astype('int'))
mean_SSE_te = mean_squared_error(y_true=X_test['rain_tomorrow'].astype('int'), y_pred=X_test['guess'].astype('int'))
print("Testing SSE:" ,SSE_te)
print("Testing Mean SSE:" ,mean_SSE_te)


Training SSE: 6720
Training Mean SSE: 0.32878320857184795
Testing SSE: 1670
Testing Mean SSE: 0.3268101761252446
